# Elastic Search Update Notebook

This notebook is used to update the elastic search index with the latest datasets

In [ ]:
!pip install --upgrade --force-reinstall git+https://github.com/rbilleci/pandora.git

In [ ]:
import boto3
import pandas as pd
import pandora
import pandora.data.age_distribution as age_dist
import pandora.data.oxford_data as oxford
import pandora.data.population as population
import pandora.data.temperatures as temperatures
import shutil
import os
import uuid
import numpy as np
from sklearn.preprocessing import RobustScaler
from pandora.data import geo, continent, country_code, working_day
from pandora import loader, encoders
from pandora.core_fields import DATE, COUNTRY_CODE
from sagemaker import get_execution_role, session
import datetime
from pathlib import Path
from elasticsearch import helpers
from requests_aws4auth import AWS4Auth
from elasticsearch import Elasticsearch, RequestsHttpConnection
from logging import INFO, basicConfig, info
import warnings

In [ ]:
# setup logging
basicConfig(level=INFO, format='%(asctime)s\t%(levelname)s\t%(filename)s\t%(message)s')
warnings.filterwarnings('ignore', category=FutureWarning)  # ignore FutureWarning from scikit learn

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_info_columns = 1000

# Download the data files

In [ ]:
import pandora.data.oxford_data_update

# Generate Dataset

In [ ]:

# determine the last date we have from the oxford data set
# which is the first day we'll begin prediction from
prediction_start_date = pd.read_csv(oxford.module.location, keep_default_na=False, na_values='')['date'].max()
prediction_start_date = datetime.datetime.strptime(prediction_start_date, '%Y-%m-%d').date()

# the data range should cover the ground truth data + the time window we are predicting into
days_to_predict = 90
start_date = datetime.date(2020, 1, 1)
end_date = prediction_start_date + datetime.timedelta(days=days_to_predict)

# the imputation window might extend far before or after
# the range we are predicting, this is so we have more data samples for imputation calculations
imputation_window_start_date =  datetime.date(2020, 1, 1)
imputation_window_end_date =  datetime.date(2021, 12, 31)

# load the data
df = loader.load(start_date,
                 end_date,
                 imputation_window_start_date,
                 imputation_window_end_date,
                 geo.module,
                 [
                     country_code.module,
                     continent.module,
                     population.module,
                     age_dist.module,
                     temperatures.module,
                     oxford.module,
                     working_day.module
                 ])

# Add computed column

In [ ]:
def compute_ma(field, window_size):
    df[f"{field}_ma_{window_size}"] = df.groupby('geo_code')[field].rolling(window_size, center=False).mean().fillna(0).reset_index(0, drop=True)

def add_working_day_tomorrow(grouped):
    grouped['working_day' + '_tomorrow'] = grouped['working_day'].copy().shift(-1).bfill().ffill()
    return grouped


def add_working_day_yesterday(grouped):
    grouped['working_day' + '_yesterday'] = grouped['working_day'].copy().shift(1).bfill().ffill()
    return grouped

def transform_column_order(df):
    df = df.reindex(sorted(df.columns), axis=1)  # Sort columns by name
    df_label = df['predicted_new_cases']
    df = df.drop(labels=['predicted_new_cases'], axis=1)
    df.insert(0, 'predicted_new_cases', df_label)
    return df

# Compute number of new cases and deaths each day
# Replace negative values (which do not make sense for these columns) with 0
df['new_cases'] = df.groupby('geo_code').confirmed_cases.diff().fillna(0)
df['new_cases'] = df['new_cases'].clip(lower=0)

# add predicted new cases
df['predicted_new_cases'] = df.groupby('geo_code').new_cases.shift(-1).fillna(0)
df['predicted_new_cases'] = df['predicted_new_cases'].clip(lower=0)

# add confirmed cases as percent of population
df['new_cases_as_percent_of_population'] = df['new_cases'] / df['population']
df['confirmed_cases_as_percent_of_population'] = df['confirmed_cases'] / df['population']

# Add moving averages
for window_size in [3, 7, 21]:
    compute_ma('new_cases', window_size)
    compute_ma('confirmed_cases', window_size)
    compute_ma('specific_humidity', window_size)    
    compute_ma('temperature', window_size)        
    compute_ma('c1_school_closing', window_size)        
    compute_ma('c2_workplace_closing', window_size)        
    compute_ma('c3_cancel_public_events', window_size)        
    compute_ma('c4_restrictions_on_gatherings', window_size)   
    compute_ma('c5_close_public_transport', window_size)        
    compute_ma('c6_stay_at_home_requirements', window_size)        
    compute_ma('c7_restrictions_on_internal_movement', window_size)        
    compute_ma('c8_international_travel_controls', window_size)   
    compute_ma('h1_public_information_campaigns', window_size)        
    compute_ma('h2_testing_policy', window_size)        
    compute_ma('h3_contact_tracing', window_size)        
    compute_ma('h6_facial_coverings', window_size)     
    compute_ma('working_day', window_size)        
    compute_ma('new_cases_as_percent_of_population', window_size)     
    compute_ma('confirmed_cases_as_percent_of_population', window_size)     

# Add working day information for tomorrow, and today
df = df.groupby('geo_code').apply(lambda group: add_working_day_tomorrow(group)).reset_index(drop=True)
df = df.groupby('geo_code').apply(lambda group: add_working_day_yesterday(group)).reset_index(drop=True)
df['npi_sum'] = df['c1_school_closing'] + df['c2_workplace_closing'] + \
                df['c3_cancel_public_events'] + df['c4_restrictions_on_gatherings'] + \
                df['c5_close_public_transport'] + df['c6_stay_at_home_requirements'] + \
                df['c7_restrictions_on_internal_movement'] + df['c8_international_travel_controls'] + \
                df['h1_public_information_campaigns'] + df['h2_testing_policy'] + \
                df['h3_contact_tracing'] + df['h6_facial_coverings']

# Drop unused columns
df = transform_column_order(df)
df = df.sort_values(['geo_code', 'date'])
df.info()

# Write out the data

In [ ]:
shutil.rmtree('temp', ignore_errors=True)
Path('temp').mkdir(parents=True, exist_ok=True)


df.to_csv('temp/01-data.csv', index=False)